In [79]:
import csv
import os
import pathlib
from glob import glob

import imutils
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import to_categorical



base_path = (pathlib.Path.home() / 'dummy_folder' / 'adience')
folds_path = str(base_path / 'folds')

AGE_BINS = [(0, 2), (4, 6), (8, 13), (15, 20), (25, 32),
            (38, 43), (48, 53), (60, 99)]

BATCH_SIZE=128

train_len = 8000
val_len = 2000

target_size=(64, 64, 3)

def age_to_bin(age):
    age = age.replace('(', '').replace(')', '').split(',')
    lower, upper = [int(x.strip()) for x in age]

    for bin_low, bin_up in AGE_BINS:
        if lower >= bin_low and upper <= bin_up:
            label = f'{bin_low}_{bin_up}'
            return label

        
def rectangle_area(r):
    return (r[2] - r[0]) * (r[3] - r[1])


def plot_face(image, age_gender, detection):
    frame_x, frame_y, frame_width, frame_height = detection
    cv2.rectangle(image,
                  (frame_x, frame_y),
                  (frame_x + frame_width,
                   frame_y + frame_height),
                  color=(0, 255, 0),
                  thickness=2)
    cv2.putText(image,
                age_gender,
                (frame_x, frame_y - 10),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.45,
                color=(0, 255, 0),
                thickness=2)

    return image


def predict(model, roi):
    roi = cv2.resize(roi, (64, 64))
    roi = roi.astype('float32') / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)

    predictions = model.predict(roi)
    return predictions


In [51]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def load_dataset(dataset_path, folds_path):
    images = []
    ages = []
    genders = []
    folds_pattern = os.path.sep.join([folds_path, '*.txt'])
    print((folds_pattern))
    for fold_path in glob(folds_pattern):
        with open(fold_path, 'r') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for line in reader:
                if ((line['age'][0] != '(') or
                        (line['gender'] not in {'m', 'f'})):
                    continue

                age_label = age_to_bin(line['age'])
                if age_label is None:
                    continue

                aligned_face_file = (f'landmark_aligned_face.'
                                     f'{line["face_id"]}.'
                                     f'{line["original_image"]}')
                image_path = os.path.sep.join([str(base_path),
                                               line["user_id"],
                                               aligned_face_file])

                image = load_img(image_path, target_size=(64, 64))
                image = img_to_array(image) / 255.0
                
                images.append(image.ravel())
                ages.append(age_label)
                genders.append(line['gender'])

    
    gender_labels = gender_enc.fit_transform(genders)
    age_labels = age_enc.fit_transform(ages)

    train_images = images[:train_len]
    val_images = images[train_len:train_len+val_len]
    test_images = images[train_len+val_len:]

    train_age_labels = age_labels[:train_len]
    val_age_labels = age_labels[train_len:train_len+val_len]
    test_age_labels = age_labels[train_len+val_len:]
    
    train_gender_labels = gender_labels[:train_len]
    val_gender_labels = gender_labels[train_len:train_len+val_len]
    test_gender_labels = gender_labels[train_len+val_len:]
    
    train_df = pd.DataFrame({'images': train_images, 'age': train_age_labels, 'gender': train_gender_labels})
    val_df = pd.DataFrame({'images': val_images, 'age': val_age_labels, 'gender': val_gender_labels})
    test_df = pd.DataFrame({'images': test_images, 'age': test_age_labels, 'gender': test_gender_labels})
                              

    return (train_df, val_df, test_df)

gender_enc = LabelEncoder()
age_enc = LabelEncoder()        
(train_df, val_df, test_df) = load_dataset(base_path, folds_path)

C:\Users\venka\dummy_folder\adience\folds\*.txt


In [62]:
val_df["age"].head, test_df["gender"].head

(<bound method NDFrame.head of 0       3
 1       3
 2       3
 3       3
 4       3
        ..
 1995    6
 1996    6
 1997    6
 1998    6
 1999    6
 Name: age, Length: 2000, dtype: int64>,
 <bound method NDFrame.head of 0      0
 1      1
 2      1
 3      0
 4      1
       ..
 824    1
 825    0
 826    0
 827    1
 828    0
 Name: gender, Length: 829, dtype: int64>)

In [66]:
gender_enc.inverse_transform([1])

array(['m'], dtype='<U1')

In [67]:
age_enc.inverse_transform([6])

array(['8_13'], dtype='<U5')

In [35]:
train_df['images']

0       [0.32941177, 0.23137255, 0.24705882, 0.7098039...
1       [0.7921569, 0.7607843, 0.5803922, 0.77254903, ...
2       [0.6745098, 0.6392157, 0.61960787, 0.7764706, ...
3       [0.2901961, 0.3254902, 0.34509805, 0.32156864,...
4       [0.078431375, 0.05490196, 0.05490196, 0.090196...
                              ...                        
9995    [0.32941177, 0.2, 0.078431375, 0.32156864, 0.1...
9996    [0.28627452, 0.35686275, 0.15294118, 0.3450980...
9997    [0.8901961, 0.9372549, 0.92941177, 0.8901961, ...
9998    [0.68235296, 0.69803923, 0.7411765, 0.7254902,...
9999    [0.8666667, 0.88235295, 0.89411765, 0.84705883...
Name: images, Length: 10000, dtype: object

In [84]:
def build_network(input_shape, classes):
    input = Input(shape=input_shape)
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same',
               kernel_initializer='he_normal')(input)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Conv2D(filters=128,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=128,
               kernel_size=(3, 3),
               kernel_initializer='he_normal',
               padding='same')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(rate=0.25)(x)

    x = Flatten()(x)
    x = Dense(units=64,
              kernel_initializer='he_normal')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.5)(x)

    x = Dense(units=64,
              kernel_initializer='he_normal')(x)
    x = ELU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.5)(x)

    output_age = Dense(units=8,
              kernel_initializer='he_normal', activation="softmax", name="output_age")(x)
    
    output_gender = Dense(units=1, activation="sigmoid", name="output_gender")(x)
    
    return Model(inputs=input, outputs=[output_age, output_gender])


In [44]:
from tensorflow.keras.utils import Sequence

class CustomDataGen(Sequence):
    
    def __init__(self, df, X_col, y_col,
                 batch_size,
                 input_size=(64, 64, 3),
                 shuffle=True):
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
        self.n_name = df[y_col['age']].nunique()
        self.n_type = df[y_col['gender']].nunique()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        image_batch = batches[self.X_col['images']]
        
        age_batch = batches[self.y_col['age']]
        gender_batch = batches[self.y_col['gender']]
        
        X_batch_t = np.asarray(image_batch)

        X_batch_list = []
        for im in X_batch_t:
            im1 = im.reshape(64, 64, 3)
            X_batch_list.append(im1)

        X_batch = np.stack([i.tolist() for i in X_batch_list])

        y0_batch = np.asarray(age_batch)
        y1_batch = np.asarray(gender_batch)
        
        print(y0_batch.shape)

        return X_batch, tuple([y0_batch, y1_batch])
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        
        X, y = self.__get_data(batches)        
        return (X, y)
    
    def __len__(self):
        return self.n // self.batch_size

In [114]:
def run_model(EPOCHS=2):
    base_path = (pathlib.Path.home() / 'dummy_folder')
    input_path = str(base_path / 'fer2013.csv')

    model = build_network((64, 64, 3), 1)
    model.compile(loss={'output_age': 'sparse_categorical_crossentropy', 'output_gender': 'binary_crossentropy'},
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    checkpoint_pattern = ('dummy_folder/model_age_gender_ep{epoch:03d}-loss{loss:.3f}'
                          '-loss{loss:.3f}.h5')
    checkpoint = ModelCheckpoint(checkpoint_pattern,
                                 monitor='loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='min')
    
    traingen = CustomDataGen(train_df,
                             X_col={'images': 'images'},
                             y_col={'age': 'age', 'gender': 'gender'},
                             batch_size=BATCH_SIZE, input_size=target_size)

    valgen = CustomDataGen(val_df,
                             X_col={'images': 'images'},
                             y_col={'age': 'age', 'gender': 'gender'},
                             batch_size=BATCH_SIZE, input_size=target_size)
                                    
    testgen = CustomDataGen(test_df,
                             X_col={'images': 'images'},
                             y_col={'age': 'age', 'gender': 'gender'},
                             batch_size=BATCH_SIZE, input_size=target_size)
                                    
    train_steps = len(traingen)
    
    model.fit(traingen,
              steps_per_epoch=train_steps,
              validation_data=valgen,
              epochs=EPOCHS,
              verbose=1,
              callbacks=[checkpoint])

    test_steps = len(testgen)
    
    loss, age_loss, gender_loss, age_acc, gender_acc = model.evaluate(testgen,
                                 steps=test_steps
                                )

    print('age accuracy is {:.2f}  - gender_accuracy is {:.2f}'.format(age_acc * 100, gender_acc * 100))
    
run_model(EPOCHS=30)

(128,)
Epoch 1/30
(128,)
43/62 [===================>..........] - ETA: 57s - loss: 3.8040 - output_age_loss: 2.9725 - output_gender_loss: 0.8315 - output_age_accuracy: 0.1386 - output_gender_accuracy: 0.5719 (128,)


62/62 [==============================] - ETA: 0s - loss: 3.6906 - output_age_loss: 2.8733 - output_gender_loss: 0.8173 - output_age_accuracy: 0.1440 - output_gender_accuracy: 0.5741(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
62/62 [==============================] - 218s 3s/step - loss: 3.6906 - output_age_loss: 2.8733 - output_gender_loss: 0.8173 - output_age_accuracy: 0.1440 - output_gender_accuracy: 0.5741 - val_loss: 3.4492 - val_output_age_loss: 2.6745 - val_output_gender_loss: 0.7747 - val_output_age_accuracy: 0.2927 - val_output_gender_accuracy: 0.5667

Epoch 00001: loss improved from inf to 3.69056, saving model to dummy_folder\model_age_gender_ep001-loss3.691-loss3.691.h5
Epoch 2/30
(128,)
21/62 [=========>....................] - ETA: 2:03 - loss: 3.1885 - output_age_loss: 2.4936 - output_gender_loss: 0.6949 - output_age_accuracy: 0.1882 - output_gender_accuracy: 0.6362(128,)


62/62 [==============================] - ETA: 0s - loss: 2.9376 - output_age_loss: 2.2738 - output_gender_loss: 0.6638 - output_age_accuracy: 0.2342 - output_gender_accuracy: 0.6536(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
62/62 [==============================] - 214s 3s/step - loss: 2.9376 - output_age_loss: 2.2738 - output_gender_loss: 0.6638 - output_age_accuracy: 0.2342 - output_gender_accuracy: 0.6536 - val_loss: 2.9253 - val_output_age_loss: 2.2721 - val_output_gender_loss: 0.6532 - val_output_age_accuracy: 0.2740 - val_output_gender_accuracy: 0.6359



Epoch 00002: loss improved from 3.69056 to 2.93759, saving model to dummy_folder\model_age_gender_ep002-loss2.938-loss2.938.h5
Epoch 3/30
(128,)
14/62 [=====>........................] - ETA: 2:21 - loss: 2.5605 - output_age_loss: 1.9599 - output_gender_loss: 0.6006 - output_age_accuracy: 0.3242 - output_gender_accuracy: 0.7115(128,)
(128,
36/62 [================>.............] - ETA: 1:16 - loss: 2.4756 - output_age_loss: 1.8905 - output_gender_loss: 0.5851 - output_age_accuracy: 0.3411 - output_gender_accuracy: 0.7094(128,)
(128,
44/62 [====================>.........] - ETA: 52s - loss: 2.4305 - output_age_loss: 1.8539 - output_gender_loss: 0.5766 - output_age_accuracy: 0.3461 - output_gender_accuracy: 0.7132(128,)


62/62 [==============================] - ETA: 0s - loss: 2.3541 - output_age_loss: 1.7956 - output_gender_loss: 0.5585 - output_age_accuracy: 0.3619 - output_gender_accuracy: 0.7215(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
62/62 [==============================] - 204s 3s/step - loss: 2.3541 - output_age_loss: 1.7956 - output_gender_loss: 0.5585 - output_age_accuracy: 0.3619 - output_gender_accuracy: 0.7215 - val_loss: 2.3390 - val_output_age_loss: 1.7907 - val_output_gender_loss: 0.5484 - val_output_age_accuracy: 0.2651 - val_output_gender_accuracy: 0.7104

Epoch 00003: loss improved from 2.93759 to 2.35406, saving model to dummy_folder\model_age_gender_ep003-loss2.354-loss2.354.h5
Epoch 4/30
(128,)
22/62 [=========>....................] - ETA: 1:57 - loss: 2.0534 - output_age_loss: 1.5649 - output_gender_loss: 0.4886 - output_age_accuracy: 0.4180 - output_gender_accuracy: 0.7656(128,)


...

62/62 [==============================] - ETA: 0s - loss: 0.4291 - output_age_loss: 0.3510 - output_gender_loss: 0.0781 - output_age_accuracy: 0.8785 - output_gender_accuracy: 0.9714(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
62/62 [==============================] - 200s 3s/step - loss: 0.4291 - output_age_loss: 0.3510 - output_gender_loss: 0.0781 - output_age_accuracy: 0.8785 - output_gender_accuracy: 0.9714 - val_loss: 2.4584 - val_output_age_loss: 1.8819 - val_output_gender_loss: 0.5766 - val_output_age_accuracy: 0.4630 - val_output_gender_accuracy: 0.8411

Epoch 00027: loss improved from 0.43761 to 0.42913, saving model to dummy_folder\model_age_gender_ep027-loss0.429-loss0.429.h5
Epoch 28/30
(128,)
29/62 [=============>................] - ETA: 1:39 - loss: 0.3893 - output_age_loss: 0.3018 - output_gender_loss: 0.0875 - output_age_accuracy: 0.8901 - output_gender_accuracy: 0.9693(128,)


62/62 [==============================] - ETA: 0s - loss: 0.3977 - output_age_loss: 0.3183 - output_gender_loss: 0.0794 - output_age_accuracy: 0.8843 - output_gender_accuracy: 0.9704(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
62/62 [==============================] - 223s 4s/step - loss: 0.3977 - output_age_loss: 0.3183 - output_gender_loss: 0.0794 - output_age_accuracy: 0.8843 - output_gender_accuracy: 0.9704 - val_loss: 2.6279 - val_output_age_loss: 1.9471 - val_output_gender_loss: 0.6808 - val_output_age_accuracy: 0.4797 - val_output_gender_accuracy: 0.8281

Epoch 00028: loss improved from 0.42913 to 0.39765, saving model to dummy_folder\model_age_gender_ep028-loss0.398-loss0.398.h5
Epoch 29/30
(128,)
 7/62 [==>...........................] - ETA: 3:04 - loss: 0.3808 - output_age_loss: 0.2786 - output_gender_loss: 0.1022 - output_age_accuracy: 0.9040 - output_gender_accuracy: 0.9609(128,)


50/62 [=======================>......] - ETA: 39s - loss: 0.3777 - output_age_loss: 0.3030 - output_gender_loss: 0.0747 - output_age_accuracy: 0.8945 - output_gender_accuracy: 0.9719(128,)


62/62 [==============================] - ETA: 0s - loss: 0.3871 - output_age_loss: 0.3100 - output_gender_loss: 0.0772 - output_age_accuracy: 0.8926 - output_gender_accuracy: 0.9708(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
62/62 [==============================] - 223s 4s/step - loss: 0.3871 - output_age_loss: 0.3100 - output_gender_loss: 0.0772 - output_age_accuracy: 0.8926 - output_gender_accuracy: 0.9708 - val_loss: 2.6085 - val_output_age_loss: 1.9306 - val_output_gender_loss: 0.6779 - val_output_age_accuracy: 0.4724 - val_output_gender_accuracy: 0.8292

Epoch 00029: loss improved from 0.39765 to 0.38714, saving model to dummy_folder\model_age_gender_ep029-loss0.387-loss0.387.h5
Epoch 30/30
(128,)
28/62 [============>.................] - ETA: 1:50 - loss: 0.3692 - output_age_loss: 0.2874 - output_gender_loss: 0.0818 - output_age_accuracy: 0.8956 - output_gender_accuracy: 0.9710(128,)


62/62 [==============================] - ETA: 0s - loss: 0.3751 - output_age_loss: 0.3018 - output_gender_loss: 0.0733 - output_age_accuracy: 0.8915 - output_gender_accuracy: 0.9730(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
(128,)
62/62 [==============================] - 225s 4s/step - loss: 0.3751 - output_age_loss: 0.3018 - output_gender_loss: 0.0733 - output_age_accuracy: 0.8915 - output_gender_accuracy: 0.9730 - val_loss: 2.4593 - val_output_age_loss: 1.8692 - val_output_gender_loss: 0.5901 - val_output_age_accuracy: 0.4469 - val_output_gender_accuracy: 0.8427

Epoch 00030: loss improved from 0.38714 to 0.37508, saving model to dummy_folder\model_age_gender_ep030-loss0.375-loss0.375.h5
(128,)
(128,)
6/6 [==============================] - 9s 1s/step - loss: 2.5800 - output_age_loss: 2.0082 - output_gender_loss: 0.5717 - output_age_accuracy: 0.4727 - output_gender_accuracy: 0.8464
age accuracy is 47.27  - gender_accuracy is 84.64

In [148]:
import cv2, imutils

model = load_model("dummy_folder/model_age_gender/model_age_gender_ep030-loss0.375-loss0.375.h5")

image = cv2.imread('dummy_folder/man-1.jpg')

det = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

image = imutils.resize(image, width=380)
copy = image.copy()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

detections = \
    det.detectMultiScale(gray,
                         scaleFactor=1.1,
                         minNeighbors=5,
                         minSize=(35, 35),
                         flags=cv2.CASCADE_SCALE_IMAGE)

if len(detections) > 0:
    detections = sorted(detections, key=rectangle_area)
    best_detection = detections[-1]

    (frame_x, frame_y,
     frame_width, frame_height) = best_detection

    roi = image[frame_y:frame_y + frame_height,
          frame_x:frame_x + frame_width]

    (age_predictions, gender_predictions) = predict(model, roi)
    
    print(age_predictions)
    age_pred = age_predictions.argmax()
    print(age_pred)
    age = age_enc.inverse_transform([age_pred])

    print(gender_predictions)
    gender_pred = 1 if (gender_predictions > 0.5) else 0
    gender = gender_enc.inverse_transform([gender_pred])

    clone = plot_face(copy,
                      f'Gender: {gender} - Age: {age}',
                      best_detection)

    cv2.imshow('Result', copy)
    cv2.waitKey(0)

[[0.00487288 0.09894756 0.19916666 0.2630655  0.39209682 0.00407013
  0.03698457 0.00079584]]
4
[[0.14382681]]


In [143]:
for i in range(6):
    print(age_enc.inverse_transform([i]))

['0_2']
['15_20']
['25_32']
['38_43']
['48_53']
['4_6']


In [145]:
print(gender_enc.inverse_transform([1]))

['m']
